In [1]:
import numpy as np
import pandas as pd

In [2]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

In [3]:
df = pd.read_csv("play_tennis.csv")

In [4]:
df.keys()

Index(['day', 'outlook', 'temp', 'humidity', 'wind', 'play'], dtype='object')

In [5]:
def calculate_overall_entropy(df):
    target = df.keys()[-1]
    
    values,counts = np.unique(df[target],return_counts=True)
    
    counts = counts/np.sum(counts)
    
    
    return np.sum(-counts*np.log2(counts))
    

In [6]:
calculate_overall_entropy(df)

0.9402859586706311

In [7]:
def calculate_entropy_of_attribute(df,attribute):
    target = df.keys()[-1]
    values_in_target = np.unique(df[target])
    
    values_in_attribute = np.unique(df[attribute])
    
    attribute_entropy = 0
    for value_in_attribute in values_in_attribute:
        overall_entropy = 0
        for value_in_target in values_in_target:
            
            num = len(df[attribute][df[attribute]==value_in_attribute][df[target]==value_in_target])
            den = len(df[attribute][df[attribute]==value_in_attribute])
            
            p = num/(den+eps)
            overall_entropy += -p*np.log2(p+eps)
        p2 = den/len(df)
        attribute_entropy += p2*overall_entropy
        
    return abs(attribute_entropy)

In [8]:
for att in df.keys()[1:-1]:
    print(f'{att},{calculate_entropy_of_attribute(df,att)}')

outlook,0.6935361388961914
temp,0.9110633930116756
humidity,0.7884504573082889
wind,0.892158928262361


# Finding the best attribute for node

In [9]:
def find_best_attribute(df):
    target = df.keys()[-1]
    IG = []
    
    for atts in df.keys()[1:-1]:
        IG.append(calculate_overall_entropy(df) - calculate_entropy_of_attribute(df,atts))
    best_attribute_to_divide = np.argmax(IG)+1
    
    return df.keys()[best_attribute_to_divide]

In [10]:
find_best_attribute(df)

'outlook'

In [21]:
def buildTree(df,tree=None):
    target = df.keys()[-1]
    
    node  = find_best_attribute(df)
    
    attValue = np.unique(df[node])
    
    if tree==None:
        tree = {}
        tree[node] ={}
    for value in attValue:
        print(df[df[node]==value].reset_index(drop=True))

In [22]:
buildTree(df)

   day   outlook  temp humidity    wind play
0   D3  Overcast   Hot     High    Weak  Yes
1   D7  Overcast  Cool   Normal  Strong  Yes
2  D12  Overcast  Mild     High  Strong  Yes
3  D13  Overcast   Hot   Normal    Weak  Yes
   day outlook  temp humidity    wind play
0   D4    Rain  Mild     High    Weak  Yes
1   D5    Rain  Cool   Normal    Weak  Yes
2   D6    Rain  Cool   Normal  Strong   No
3  D10    Rain  Mild   Normal    Weak  Yes
4  D14    Rain  Mild     High  Strong   No
   day outlook  temp humidity    wind play
0   D1   Sunny   Hot     High    Weak   No
1   D2   Sunny   Hot     High  Strong   No
2   D8   Sunny  Mild     High    Weak   No
3   D9   Sunny  Cool   Normal    Weak  Yes
4  D11   Sunny  Mild   Normal  Strong  Yes
